In [3]:
#CNN

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Data augmentation (you can customize this)
    datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Feature extraction using a CNN
def create_cnn_feature_extractor(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
    ])
    return model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
feature_extractor = create_cnn_feature_extractor(input_shape)
feature_extractor.build(input_shape)
feature_extractor.compile()

# Extract features
x_train_features = feature_extractor.predict(np.array(x_train))
x_test_features = feature_extractor.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features, y_train)
    y_pred = classifier.predict(x_test_features)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")



19/19 [==============================] - 2s 82ms/step
AdaBoost accuracy: 0.8833333333333333
KNN accuracy: 0.9083333333333333
RF accuracy: 0.9266666666666666
SVM accuracy: 0.8816666666666667
Softmax accuracy: 0.965

Average Accuracy: 0.913


In [4]:
#Xception


import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Feature extraction using Xception
def create_xception_feature_extractor(input_shape):
    base_model = keras.applications.Xception(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling='avg'
    )
    return base_model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
feature_extractor = create_xception_feature_extractor(input_shape)

# Extract features
x_train_features = feature_extractor.predict(np.array(x_train))
x_test_features = feature_extractor.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features, y_train)
    y_pred = classifier.predict(x_test_features)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

19/19 [==============================] - 10s 521ms/step
AdaBoost accuracy: 0.91
KNN accuracy: 0.9683333333333334
RF accuracy: 0.9533333333333334
SVM accuracy: 0.965
Softmax accuracy: 0.9816666666666667

Average Accuracy: 0.9556666666666667


In [1]:
#InceptionV3

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'
# Data preprocessing and augmentation
image_size = (150, 150)  # InceptionV3's required input size
batch_size = 16

# Function to preprocess data
def preprocess_data(data):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess the loaded data
    data = preprocess_data(data)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Feature extraction using InceptionV3
def create_inceptionv3_feature_extractor(input_shape):
    base_model = keras.applications.InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
feature_extractor = create_inceptionv3_feature_extractor(input_shape)

# Extract features
x_train_features = feature_extractor.predict(np.array(x_train))
x_test_features = feature_extractor.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

accuracies = []

# Train and evaluate each classifier
for name, classifier in classifiers.items():
    classifier.fit(x_train_features.reshape(x_train_features.shape[0], -1), y_train)
    y_pred = classifier.predict(x_test_features.reshape(x_test_features.shape[0], -1))
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

average_accuracy = np.mean(accuracies)
print(f"Average accuracy of classifiers: {average_accuracy}")


19/19 [==============================] - 9s 394ms/step
AdaBoost accuracy: 0.94
KNN accuracy: 0.9816666666666667
RF accuracy: 0.965
SVM accuracy: 0.975
Softmax accuracy: 0.9783333333333334
Average accuracy of classifiers: 0.968


In [2]:
#ResNet50


import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size = (224, 224)  # Image size for ResNet50
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Resize images for ResNet50 input size
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization for ResNet50
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Feature extraction using ResNet50
preprocessed_images = np.array(data)
preprocessed_images = keras.applications.resnet50.preprocess_input(preprocessed_images)

resnet_model = keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size[0], image_size[1], 3)
)

x_train_features = resnet_model.predict(np.array(x_train))
x_test_features = resnet_model.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features.reshape(len(x_train), -1), y_train)
    y_pred = classifier.predict(x_test_features.reshape(len(x_test), -1))
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate and show average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage accuracy of classifiers: {average_accuracy}")


19/19 [==============================] - 26s 1s/step
AdaBoost accuracy: 0.8583333333333333
KNN accuracy: 0.905
RF accuracy: 0.9216666666666666
SVM accuracy: 0.77
Softmax accuracy: 0.97

Average accuracy of classifiers: 0.885


C:\Users\mohiu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
#EfficientNetB0

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    data = [cv2.resize(img, image_size) for img in data]
    data = np.array(data)
    data = preprocess_input(data)  # EfficientNet's preprocessing

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

# Extract features using EfficientNetB0
x_train_features = efficientnet_model.predict(np.array(x_train))
x_test_features = efficientnet_model.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features.reshape(x_train_features.shape[0], -1), y_train)
    y_pred = classifier.predict(x_test_features.reshape(x_test_features.shape[0], -1))
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average accuracy of classifiers: {average_accuracy}")


19/19 [==============================] - 6s 248ms/step
AdaBoost accuracy: 0.9433333333333334
KNN accuracy: 0.985
RF accuracy: 0.9733333333333334
SVM accuracy: 0.995
Softmax accuracy: 0.9983333333333333
Average accuracy of classifiers: 0.9789999999999999


In [4]:
#VGG19

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size = (224, 224)  # VGG19 input size
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Preprocess for VGG19 input
x_train = np.array(x_train)
x_test = np.array(x_test)

# Normalize the images for VGG19
x_train = keras.applications.vgg19.preprocess_input(x_train)
x_test = keras.applications.vgg19.preprocess_input(x_test)

# Load the VGG19 model
vgg_model = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=image_size + (3,))

# Extract features
x_train_features = vgg_model.predict(x_train)
x_test_features = vgg_model.predict(x_test)

# Flatten extracted features
x_train_features = x_train_features.reshape(x_train_features.shape[0], -1)
x_test_features = x_test_features.reshape(x_test_features.shape[0], -1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
for name, classifier in classifiers.items():
    classifier.fit(x_train_features, y_train)
    y_pred = classifier.predict(x_test_features)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} accuracy: {accuracy}")

    

average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average accuracy of classifiers: {average_accuracy}")


19/19 [==============================] - 88s 5s/step
AdaBoost accuracy: 0.8233333333333334
KNN accuracy: 0.845
RF accuracy: 0.9183333333333333
SVM accuracy: 0.7416666666666667
Softmax accuracy: 0.9066666666666666
Average accuracy of classifiers: 0.9789999999999999


C:\Users\mohiu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [1]:
#InceptionV3 + VGG19

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size_vgg = (224, 224)  # VGG19 input size
image_size_xception = (128, 128)
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)

            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)

    return data, labels

# Load the VGG19 dataset
data_vgg, labels_vgg = load_dataset(original_dataset_path, image_size_vgg)

# Split the VGG19 data into training and testing sets
x_train_vgg, x_test_vgg, y_train_vgg, y_test_vgg = train_test_split(data_vgg, labels_vgg, test_size=0.2, random_state=42)

# Preprocess for VGG19 input
x_train_vgg = np.array(x_train_vgg)
x_test_vgg = np.array(x_test_vgg)
x_train_vgg = keras.applications.vgg19.preprocess_input(x_train_vgg)
x_test_vgg = keras.applications.vgg19.preprocess_input(x_test_vgg)

# Load the VGG19 model
vgg_model = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=image_size_vgg + (3,))

# Extract features
x_train_features_vgg = vgg_model.predict(x_train_vgg)
x_test_features_vgg = vgg_model.predict(x_test_vgg)

# Flatten extracted features
x_train_features_vgg = x_train_features_vgg.reshape(x_train_features_vgg.shape[0], -1)
x_test_features_vgg = x_test_features_vgg.reshape(x_test_features_vgg.shape[0], -1)

# Load the Xception dataset
data_xception, labels_xception = load_dataset(original_dataset_path, image_size_xception)

# Split the Xception data into training and testing sets
x_train_xception, x_test_xception, y_train_xception, y_test_xception = train_test_split(
    data_xception, labels_xception, test_size=0.2, random_state=42
)

# Feature extraction using Xception
def create_xception_feature_extractor(input_shape):
    base_model = keras.applications.Xception(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling='avg'
    )
    return base_model

# Define the feature extractor model for Xception
input_shape_xception = (image_size_xception[0], image_size_xception[1], 3)  # 3 for RGB channels
feature_extractor_xception = create_xception_feature_extractor(input_shape_xception)

# Extract features for Xception
x_train_features_xception = feature_extractor_xception.predict(np.array(x_train_xception))
x_test_features_xception = feature_extractor_xception.predict(np.array(x_test_xception))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Concatenate the features from both models
x_train_features_combined = np.concatenate((x_train_features_vgg, x_train_features_xception), axis=1)
x_test_features_combined = np.concatenate((x_test_features_vgg, x_test_features_xception), axis=1)

# Train and evaluate each classifier
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features_combined, y_train_vgg)
    y_pred = classifier.predict(x_test_features_combined)
    accuracy = accuracy_score(y_test_vgg, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")



19/19 [==============================] - 10s 479ms/step
AdaBoost accuracy: 0.9266666666666666
KNN accuracy: 0.9183333333333333
RF accuracy: 0.9483333333333334
SVM accuracy: 0.82
Softmax accuracy: 0.98

Average Accuracy: 0.9186666666666667


C:\Users\mohiu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [1]:
#InceptionV3 + EfficientNetB0

import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'

# Data preprocessing and augmentation
image_size_efficientnet = (128, 128)  # EfficientNetB0's required input size
image_size_inception = (128, 128)  # InceptionV3's required input size
batch_size = 32

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)

    return data, labels

# Load the EfficientNetB0 dataset
data_efficientnet, labels_efficientnet = load_dataset(original_dataset_path, image_size_efficientnet)

# Split the EfficientNetB0 data into training and testing sets
x_train_efficientnet, x_test_efficientnet, y_train_efficientnet, y_test_efficientnet = train_test_split(
    data_efficientnet, labels_efficientnet, test_size=0.2, random_state=42
)

# Load the InceptionV3 dataset
data_inception, labels_inception = load_dataset(original_dataset_path, image_size_inception)

# Split the InceptionV3 data into training and testing sets
x_train_inception, x_test_inception, y_train_inception, y_test_inception = train_test_split(
    data_inception, labels_inception, test_size=0.2, random_state=42
)

# Feature extraction using EfficientNetB0
def create_efficientnet_feature_extractor(input_shape):
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Feature extraction using InceptionV3
def create_inceptionv3_feature_extractor(input_shape):
    base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Define the feature extractor model for EfficientNetB0
input_shape_efficientnet = (image_size_efficientnet[0], image_size_efficientnet[1], 3)  # 3 for RGB channels
feature_extractor_efficientnet = create_efficientnet_feature_extractor(input_shape_efficientnet)

# Extract features for EfficientNetB0
x_train_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_train_efficientnet))
x_test_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_test_efficientnet))

# Define the feature extractor model for InceptionV3
input_shape_inception = (image_size_inception[0], image_size_inception[1], 3)  # 3 for RGB channels
feature_extractor_inception = create_inceptionv3_feature_extractor(input_shape_inception)

# Extract features for InceptionV3
x_train_features_inception = feature_extractor_inception.predict(np.array(x_train_inception))
x_test_features_inception = feature_extractor_inception.predict(np.array(x_test_inception))

# Reshape the feature arrays before concatenating
x_train_features_efficientnet_reshaped = x_train_features_efficientnet.reshape((x_train_features_efficientnet.shape[0], -1))
x_test_features_efficientnet_reshaped = x_test_features_efficientnet.reshape((x_test_features_efficientnet.shape[0], -1))

x_train_features_inception_reshaped = x_train_features_inception.reshape((x_train_features_inception.shape[0], -1))
x_test_features_inception_reshaped = x_test_features_inception.reshape((x_test_features_inception.shape[0], -1))

# Concatenate the reshaped features from both models
x_train_features_combined = np.concatenate((x_train_features_efficientnet_reshaped, x_train_features_inception_reshaped), axis=1)
x_test_features_combined = np.concatenate((x_test_features_efficientnet_reshaped, x_test_features_inception_reshaped), axis=1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier on the combined features
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features_combined, y_train_efficientnet)
    y_pred = classifier.predict(x_test_features_combined)
    accuracy = accuracy_score(y_test_efficientnet, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")


19/19 [==============================] - 5s 220ms/step
AdaBoost accuracy: 0.9383333333333334
KNN accuracy: 0.9633333333333334
RF accuracy: 0.9516666666666667
SVM accuracy: 0.96
Softmax accuracy: 0.9866666666666667

Average Accuracy: 0.96


In [2]:
#EfficientNetB0 + VGG19

import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0, VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
 
# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'
 
# Data preprocessing and augmentation
image_size_efficientnet = (128, 128)  # EfficientNetB0's required input size
image_size_vgg19 = (128, 128)  # VGG19's required input size
batch_size = 32
 
# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]
 
    # Data normalization
    data = np.array(data) / 255.0
 
    return data, labels
 
# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []
 
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")
 
    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)
 
    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)
 
    return data, labels
 
# Load the EfficientNetB0 dataset
data_efficientnet, labels_efficientnet = load_dataset(original_dataset_path, image_size_efficientnet)
 
# Split the EfficientNetB0 data into training and testing sets
x_train_efficientnet, x_test_efficientnet, y_train_efficientnet, y_test_efficientnet = train_test_split(
    data_efficientnet, labels_efficientnet, test_size=0.2, random_state=42
)
 
# Load the VGG19 dataset
data_vgg19, labels_vgg19 = load_dataset(original_dataset_path, image_size_vgg19)
 
# Split the VGG19 data into training and testing sets
x_train_vgg19, x_test_vgg19, y_train_vgg19, y_test_vgg19 = train_test_split(
    data_vgg19, labels_vgg19, test_size=0.2, random_state=42
)
 
# Feature extraction using EfficientNetB0
def create_efficientnet_feature_extractor(input_shape):
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model
 
# Feature extraction using VGG19
def create_vgg19_feature_extractor(input_shape):
    base_model = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False
    return base_model
 
# Define the feature extractor model for EfficientNetB0
input_shape_efficientnet = (image_size_efficientnet[0], image_size_efficientnet[1], 3)
feature_extractor_efficientnet = create_efficientnet_feature_extractor(input_shape_efficientnet)
 
# Extract features for EfficientNetB0
x_train_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_train_efficientnet))
x_test_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_test_efficientnet))
 
# Define the feature extractor model for VGG19
input_shape_vgg19 = (image_size_vgg19[0], image_size_vgg19[1], 3)
feature_extractor_vgg19 = create_vgg19_feature_extractor(input_shape_vgg19)
 
# Extract features for VGG19
x_train_features_vgg19 = feature_extractor_vgg19.predict(np.array(x_train_vgg19))
x_test_features_vgg19 = feature_extractor_vgg19.predict(np.array(x_test_vgg19))
 
# Reshape the feature arrays before concatenating
x_train_features_efficientnet_reshaped = x_train_features_efficientnet.reshape((x_train_features_efficientnet.shape[0], -1))
x_test_features_efficientnet_reshaped = x_test_features_efficientnet.reshape((x_test_features_efficientnet.shape[0], -1))
 
x_train_features_vgg19_reshaped = x_train_features_vgg19.reshape((x_train_features_vgg19.shape[0], -1))
x_test_features_vgg19_reshaped = x_test_features_vgg19.reshape((x_test_features_vgg19.shape[0], -1))
 
# Concatenate the reshaped features from both models
x_train_features_combined = np.concatenate((x_train_features_efficientnet_reshaped, x_train_features_vgg19_reshaped), axis=1)
x_test_features_combined = np.concatenate((x_test_features_efficientnet_reshaped, x_test_features_vgg19_reshaped), axis=1)
 
# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}
 
# Train and evaluate each classifier on the combined features
accuracies = []
for name, classifier in classifiers.items():
    classifier.fit(x_train_features_combined, y_train_efficientnet)
    y_pred = classifier.predict(x_test_features_combined)
    accuracy = accuracy_score(y_test_efficientnet, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")
 
# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

19/19 [==============================] - 29s 2s/step
AdaBoost accuracy: 0.935
KNN accuracy: 0.9783333333333334
RF accuracy: 0.955
SVM accuracy: 0.88
Softmax accuracy: 0.985

Average Accuracy: 0.9466666666666667


In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
from tensorflow.keras.applications import InceptionV3, VGG19
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
 
# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/Brain MRI'
 
# Data preprocessing and augmentation
image_size_inception = (128, 128)  # InceptionV3's required input size
image_size_efficientnet = (128, 128)  # EfficientNetB0's required input size
image_size_vgg = (128, 128)  # VGG19 input size
batch_size = 32
 
# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]
 
    # Data normalization
    data = np.array(data) / 255.0
 
    return data, labels
 
# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []
 
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")
 
    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)
 
    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)
 
    return data, labels
 
# Load the InceptionV3 dataset
data_inception, labels_inception = load_dataset(original_dataset_path, image_size_inception)
 
# Split the InceptionV3 data into training and testing sets
x_train_inception, x_test_inception, y_train_inception, y_test_inception = train_test_split(
    data_inception, labels_inception, test_size=0.2, random_state=42
)
 
# Load the EfficientNetB0 dataset
data_efficientnet, labels_efficientnet = load_dataset(original_dataset_path, image_size_efficientnet)
 
# Split the EfficientNetB0 data into training and testing sets
x_train_efficientnet, x_test_efficientnet, y_train_efficientnet, y_test_efficientnet = train_test_split(
    data_efficientnet, labels_efficientnet, test_size=0.2, random_state=42
)
 
# Load the VGG19 dataset
data_vgg, labels_vgg = load_dataset(original_dataset_path, image_size_vgg)
 
# Split the VGG19 data into training and testing sets
x_train_vgg, x_test_vgg, y_train_vgg, y_test_vgg = train_test_split(data_vgg, labels_vgg, test_size=0.2, random_state=42)
 
# Preprocess for VGG19 input
x_train_vgg = np.array(x_train_vgg)
x_test_vgg = np.array(x_test_vgg)
x_train_vgg = keras.applications.vgg19.preprocess_input(x_train_vgg)
x_test_vgg = keras.applications.vgg19.preprocess_input(x_test_vgg)
 
# Load the InceptionV3 model for feature extraction
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=image_size_inception + (3,))
x_train_features_inception = inception_model.predict(np.array(x_train_inception))
x_test_features_inception = inception_model.predict(np.array(x_test_inception))
 
# Load the EfficientNetB0 model for feature extraction
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=image_size_efficientnet + (3,))
x_train_features_efficientnet = efficientnet_model.predict(np.array(x_train_efficientnet))
x_test_features_efficientnet = efficientnet_model.predict(np.array(x_test_efficientnet))
 
# Load the VGG19 model for feature extraction
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=image_size_vgg + (3,))
x_train_features_vgg = vgg_model.predict(x_train_vgg)
x_test_features_vgg = vgg_model.predict(x_test_vgg)
 
# Flatten the extracted features
x_train_features_inception = x_train_features_inception.reshape((x_train_features_inception.shape[0], -1))
x_test_features_inception = x_test_features_inception.reshape((x_test_features_inception.shape[0], -1))
 
x_train_features_efficientnet = x_train_features_efficientnet.reshape((x_train_features_efficientnet.shape[0], -1))
x_test_features_efficientnet = x_test_features_efficientnet.reshape((x_test_features_efficientnet.shape[0], -1))
 
x_train_features_vgg = x_train_features_vgg.reshape((x_train_features_vgg.shape[0], -1))
x_test_features_vgg = x_test_features_vgg.reshape((x_test_features_vgg.shape[0], -1))
 
# Concatenate the features from all three models
x_train_features_combined = np.concatenate((x_train_features_inception, x_train_features_efficientnet, x_train_features_vgg), axis=1)
x_test_features_combined = np.concatenate((x_test_features_inception, x_test_features_efficientnet, x_test_features_vgg), axis=1)
 
# Initialize an ensemble classifier
ensemble_classifier = RandomForestClassifier()
 
# Train the ensemble classifier
ensemble_classifier.fit(x_train_features_combined, y_train_inception)
 
# Evaluate the ensemble classifier
y_pred_ensemble = ensemble_classifier.predict(x_test_features_combined)
accuracy_ensemble = accuracy_score(y_test_inception, y_pred_ensemble)
print(f"\nEnsemble Classifier Accuracy: {accuracy_ensemble}")

19/19 [==============================] - 28s 1s/step

Ensemble Classifier Accuracy: 0.955
